In [1]:
%load_ext autoreload
%autoreload 2
from stresstest.util import load_json
import os
from loguru import logger
logger.remove(0)
print(os.getcwd())
predictions_albert = load_json('../data/predictions-albert-50-5.json')
gold = load_json('../data/stresstest-ssq-50-5.json')

/home/viktor/workspace/mrc-stresstest/notebooks


Sort by reasoning

In [2]:
from collections import defaultdict
by_reasoning = []
for sample, prediction in zip(gold, predictions_albert):
    by_reasoning_by_sample = defaultdict(list)
    for item in sample:
        passage_id, passage, qas = item['id'], item['passage'], item['qas']
        for qa in qas:
            by_reasoning_by_sample[qa['reasoning']].append((
                passage_id,
                passage,
                qa['id'],
                qa['question'],
                qa['answer']
            ))
    by_reasoning.append(by_reasoning_by_sample)

In [3]:
from stresstest.eval_utils import em, f1, get_mean_var_ci
for reasoning_type in by_reasoning[0].keys():
    print(f"REASONING TYPE: {reasoning_type}")
    per_type_em = []
    per_type_f1 = []
    for i, sample in enumerate(by_reasoning):
        per_type_em.append(em(sample[reasoning_type], predictions_albert[i]))
        per_type_f1.append(f1(sample[reasoning_type], predictions_albert[i]))
    print("EM", get_mean_var_ci(per_type_em))
    print("F1", get_mean_var_ci(per_type_f1))

REASONING TYPE: retrieval
EM (0.426262457872365, 5.864745486148903e-05, 0.009508864829570862)
F1 (0.5542834405448348, 0.0001558646449868076, 0.015501648904462125)
REASONING TYPE: ordering-easy
EM (0.1784658373680771, 0.00015457000536614292, 0.015437134924665094)
F1 (0.2296049476170948, 0.00012810747801242622, 0.01405372107409324)


In [4]:
from collections import defaultdict
by_question_type = []
for sample, prediction in zip(gold, predictions_albert):
    by_question_type_by_sample = defaultdict(list)
    for item in sample:
        passage_id, passage, qas = item['id'], item['passage'], item['qas']
        for qa in qas:
            by_question_type_by_sample[qa['type']].append((
                passage_id,
                passage,
                qa['id'],
                qa['question'],
                qa['answer']
            ))
    by_question_type.append(by_question_type_by_sample)

In [5]:

from stresstest.eval_utils import em, f1, get_mean_var_ci
for question_type in by_question_type[0].keys():
    print(f"QUESTION TYPE: {question_type}")
    per_type_em = []
    per_type_f1 = []
    for i, sample in enumerate(by_question_type):
        per_type_em.append(em(sample[question_type], predictions_albert[i]))
        per_type_f1.append(f1(sample[question_type], predictions_albert[i]))
    print("EM", get_mean_var_ci(per_type_em))
    print("F1", get_mean_var_ci(per_type_f1))



QUESTION TYPE: direct
EM (0.28098236409472954, 2.6265362377727757e-05, 0.006363496177190821)
F1 (0.34975943222385253, 0.00014474108515162874, 0.014938259556837119)
QUESTION TYPE: overall
EM (0.3610515827603925, 0.00015044715350948283, 0.015229865804232906)
F1 (0.6226584277182343, 0.0011321592577466107, 0.041778981662367085)


EM misses


In [9]:
from stresstest.util import  sample_iter
for sample, predictions in zip(gold, predictions_albert):
    for story_id, story, question_id, question, answer, qa in sample_iter(sample, True):
        answer = answer or ''
        prediction = predictions[story_id][question_id]
        if "".join(str(answer).lower().split()) != "".join(str(prediction).lower().split()) and not 'ordering' in qa['reasoning']:
            print(20*"====")
            print("Question:", question)
            print("Passage: ", story)
            print("Answer:", answer)
            print("Prediction:", prediction)

Question: When was the 1 th goal scored?
Passage:  The match started as Elsie Vajgrt scored on the 67 th minute scoring the ball in the right corner after she ran 14 metres . But FC Duckburg retaliated as Laura Owens slotted in a goal for FC Duckburg . Nina Green , one of Arctic Whalesharks 's better performers today, constituted a counter strike as she scored on the 87 th minute and for her 10 th league goal of the season it was spectacular . Nina Green scored on the 88 th minute to continue where Arctic Whalesharks left off when she dribbled 2 metres on the flank and intercepted FC Duckburg 's goalkeeper's goal kick before drilling in the ball off the post and in the back of the net . Then Elsie Vajgrt disappointing the crowd with an unnecessary action , felling Laura Owens near the 6-yard-area .
Answer: 67
Prediction: 67 th minute
Question: Who committed the 1 th foul?
Passage:  The match started as Elsie Vajgrt scored on the 67 th minute scoring the ball in the right corner after s